In [3]:
from dask import dataframe as df
from dask import delayed 
import dask
import nltk
import re,string,unicodedata


In [4]:
#source https://examples.dask.org/machine-learning/text-vectorization.html
from dask.distributed import Client, progress

client = Client(n_workers=16, threads_per_worker=2, memory_limit='3GB')
client

/opt/conda/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 30755 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:30755/status,
Dashboard: http://127.0.0.1:30755/status,Workers: 16
Total threads: 32,Total memory: 44.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33207,Workers: 16
Dashboard: http://127.0.0.1:30755/status,Total threads: 32
Started: Just now,Total memory: 44.70 GiB
Comm: tcp://127.0.0.1:24951,Total threads: 2
Dashboard: http://127.0.0.1:27501/status,Memory: 2.79 GiB
Nanny: tcp://127.0.0.1:38855,


distributed.utils - ERROR - 'start'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/distributed/utils.py", line 695, in log_errors
    yield
  File "/opt/conda/lib/python3.9/site-packages/distributed/dashboard/components/shared.py", line 285, in update
    ts = metadata["keys"][self.key]
KeyError: 'start'
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fa6284b15b0>>, <Task finished name='Task-2913' coro=<_needs_document_lock.<locals>._needs_document_lock_wrapper() done, defined at /opt/conda/lib/python3.9/site-packages/bokeh/server/session.py:77> exception=KeyError('start')>)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/opt/conda/lib/python3.9/site-packages/tornado/ioloop.py", line 765, in _discard_future_result
    future.resul

In [10]:
#client.restart()

Future exception was never retrieved
future: <Future finished exception=CommClosedError("Exception while trying to call remote method 'restart' before comm was established.")>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/distributed/comm/tcp.py", line 211, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/distributed/core.py", line 846, in send_recv_from_rpc
    result = await send_recv(comm=comm, op=key, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/distributed/core.py", line 691, in send_recv
    response = await comm.read(deserializers=deserializers)
  File "/opt/conda/lib/python3.9/site-packages/distributed/comm/tcp.py", line 227, in read
    convert_stream_closed_error(self, e)
  File "/opt/conda/lib/python3.9/site-pack

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 32,Total memory: 44.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33013,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: 5 minutes ago,Total memory: 44.70 GiB
Comm: tcp://127.0.0.1:39199,Total threads: 2
Dashboard: http://127.0.0.1:37897/status,Memory: 2.79 GiB
Nanny: tcp://127.0.0.1:27453,


In [5]:
import dask.dataframe as dd
import pandas as pd

#df = dd.from_pandas(pd.DataFrame({"review": bunch.data, "target": bunch.target}),npartitions=25)
df=dd.from_pandas(pd.read_csv("IMDB Dataset.csv"),npartitions=25)
df

,review,sentiment
npartitions=25,,
0,object,object
2000,...,...
...,...,...
48000,...,...
49999,...,...


Preprocesing oz. čiščenje besedila

In [6]:
#Text cleaning
@delayed
def para_cleaning(x):
    x = re.sub('[,\.!?:()"]', '', x)
    x = re.sub('<.*?>', ' ', x)
    x = re.sub('http\S+', ' ', x)
    x = re.sub('[^a-zA-Z0-9]', ' ', x)
    x = re.sub('\s+', ' ', x)
    return x.lower().strip()

# tokennize
@delayed
def tokennize(x):
    words=nltk.tokenize.word_tokenize(x)
    return words
#Odstranjevanj stopwords


@delayed
def para_sw_remove(x):
    words = nltk.tokenize.word_tokenize(x)
    filtered_list = [word for word in words if word not in sw_set]
    return ' '.join(filtered_list)

In [7]:
#Izbira stopwords kot so am, not, a, the ...
sw_set = set(nltk.corpus.stopwords.words('english'))

In [8]:
#@delayed
def clean_text2(df):

    df["review"] = df.review.map(lambda review:review.lower()).map(para_cleaning).map(para_sw_remove)
    return df

Ocene pred čiščenjem besedila

In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
%time df=df.map_partitions(clean_text2,meta=df.compute()).compute()

CPU times: user 12 s, sys: 2.79 s, total: 14.8 s
Wall time: 17.9 s


In [ ]:
%time df['review']= dask.compute(*df['review'])

Ocene po čiščenju besedila

In [20]:
df.head()

AttributeError: 'tuple' object has no attribute 'head'

In [14]:
print(df.head().loc[0, 'review'][:500])

one reviewers mentioned watching 1 oz episode hooked right exactly happened first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians


In [21]:
import dask_ml.feature_extraction.text

vect = dask_ml.feature_extraction.text.HashingVectorizer()
X = vect.fit_transform(df['review'])
X

<50000x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 4880750 stored elements in Compressed Sparse Row format>

In [14]:
seti=['positive','negative']
seti

['positive', 'negative']

In [17]:
#!pip install dask-ml

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 148 kB 4.4 MB/s eta 0:00:01


In [18]:
import numpy as np
from dask_ml import preprocessing

le = preprocessing.LabelEncoder()

y=sentiment_data=le.fit_transform(df['sentiment'])
#y

In [19]:
import numpy as np
import sklearn.linear_model
import sklearn.pipeline

import dask_ml.wrappers

In [22]:
sgd = sklearn.linear_model.SGDClassifier(
    tol=1e-3
)
clf = dask_ml.wrappers.Incremental(
    sgd, scoring='accuracy', assume_equal_chunks=True
)
pipe = sklearn.pipeline.make_pipeline(vect, clf)

In [23]:
%%time 
pipe.fit(df['review'], y,
         incremental__classes=[0, 1]);

CPU times: user 8.08 s, sys: 311 ms, total: 8.39 s
Wall time: 7.27 s


Pipeline(steps=[('hashingvectorizer', HashingVectorizer()),
                ('incremental',
                 Incremental(estimator=SGDClassifier(), scoring='accuracy'))])

In [41]:
%%time
predictions = pipe.predict(df['review'])
predictions

CPU times: user 7.18 s, sys: 181 ms, total: 7.36 s
Wall time: 7.11 s


array([1, 1, 1, ..., 1, 0, 0])

In [38]:
%%time
accuracy_score=dask_ml.metrics.accuracy_score(y, predictions)
accuracy_score

CPU times: user 8.89 ms, sys: 2 ms, total: 10.9 ms
Wall time: 9.33 ms


0.89748

In [37]:
import random
import pandas as pd
df_original = pd.read_csv('IMDB Dataset.csv')
test_rev=df['review']
# Izbiranje naključnega reviewa

idx_test = random.randint(0, len(y)-1)
idx_original = test_rev.index[idx_test]
(actual_rev, actual_sent) = df_original.iloc[idx_original]

# Primerjanje original ocene z "naučeno" oceno naključno izbranega 
prediction = predictions[idx_test]
prediction=prediction*accuracy_score if prediction<0.5 else prediction*accuracy_score

prediction_sent = 'positive' if prediction >= 0.5 else 'negative'
prediction=accuracy_score
probability = round(accuracy_score if prediction >= 0.5 else 1-prediction, 2)

# Izpis

print('\033[1m' + 'Review #%d:' % idx_original + '\033[0m' + '\n', actual_rev, '\n')
print('\033[1m' + 'Napovedana ocena:' + '\033[0m', prediction_sent, '(z verjetnostjo %.2f)' % probability, '\n')
print('\033[1m' + 'Dejanska ocena:' + '\033[0m', actual_sent)

Review #47435:
 Please note that I haven't seen the film since I discovered it in 2007, and my town is smaller and doesn't carry it. However, I really want to say something about it. I'm actually doing research for university on the title character Richard Maurice Bucke and would like to point out that the person they based the main character on was in reality completely different!!! Hollywood's ideas of people and artistic license granted, the real Dr. Bucke totally endorsed hysterectomies to cure insanity in women, and would never have practiced anything as liberal as represented in the film. I think it's laughable to see various film critics who write for legitimate newspapers who say this film has some historical basis! The only actual fact I can see is the friendship between Dr. Bucke and Walt Whitman. Please don't waste your time on a film with such a disregard to the horror that real women experienced at the hands of this doctor who has now been glorified by the film industry. 


review       Starts off with Fulci playing a version of him...
sentiment                                             negative
Name: 5702, dtype: object